# Evaluating a Strategy using Bootstrap Metrics

Bootstrap metrics can help us to more thoroughly evaluate a trading strategy, as we will see below.

[In the last notebook](https://pybroker.com/en/latest/notebooks/2.%20Backtesting%20a%20Strategy.html), we wrote a trading strategy that we backtested. Here is the implementation again:

In [1]:
import pybroker
from pybroker import Strategy, StrategyConfig, YFinance

pybroker.enable_data_source_cache('my_strategy')

def buy_low(ctx):
    if ctx.long_pos():
        return
    if len(ctx.low) >= 2 and ctx.close[-1] < ctx.low[-2]:
        ctx.buy_shares = ctx.calc_target_shares(0.25)
        ctx.buy_limit_price = ctx.close[-1] - 0.01
        ctx.hold_bars = 3
        
def short_high(ctx):
    if ctx.short_pos():
        return
    if len(ctx.high) >= 2 and ctx.close[-1] > ctx.high[-2]:
        ctx.sell_shares = 100
        ctx.hold_bars = 2

And as before, we configure a new [Strategy](https://pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy) instance:

In [2]:
config = StrategyConfig(initial_cash=500_000, bootstrap_sample_size=100)
strategy = Strategy(YFinance(), '3/1/2017', '3/1/2022', config)

This time, the ```Strategy``` is configured with a [bootstrap_sample_size](https://pybroker.com/en/latest/reference/pybroker.config.html#pybroker.config.StrategyConfig.bootstrap_sample_size) of ```100``` (the default is ```1_000```). Next, the ```Strategy``` is backtested again, but now with bootstrap metrics enabled by default:

In [3]:
strategy.add_execution(buy_low, ['AAPL', 'MSFT'])
strategy.add_execution(short_high, ['TSLA'])
result = strategy.backtest()
result.metrics_df

Backtesting: 2017-03-01 00:00:00 to 2022-03-01 00:00:00

Loaded cached bar data.

Test split: 2017-03-01 05:00:00 to 2022-02-28 05:00:00


100% (1259 of 1259) |####################| Elapsed Time: 0:00:00 Time:  0:00:00



Calculating bootstrap metrics: sample_size=100, samples=10000...
Calculated bootstrap metrics: 0:00:03 

Finished backtest: 0:00:06


,name,value
0,trade_count,388.000000
1,initial_market_value,500000.000000
2,end_market_value,693111.870000
3,total_pnl,165740.200000
4,total_return_pct,33.148040
5,total_profit,403511.080000
6,total_loss,-237770.880000
7,max_drawdown,-56721.600000
8,max_drawdown_pct,-7.908429
9,win_rate,0.525773


When looking at ```total_pnl``` above, it appears that we have successfully implemented a profitable trading strategy on our first attempt! But how can we be so sure that those results are repeatable and were not just a fluke? We can gain more confidence in our results by computing metrics using the boostrap method.

The basic idea behind the bootstrap method is to repeatedly compute a metric on random samples drawn from the backtest results. Then the metric is computed on each random sample and the average is taken. By computing the metric on thousands of random samples, a more robust and accurate estimate of the metric is obtained.

## Confidence Intervals

**PyBroker** uses the bootstrap method to compute confidence intervals for the [Profit Factor](https://pybroker.com/en/latest/reference/pybroker.eval.html#pybroker.eval.EvalMetrics.profit_factor) and [Sharpe Ratio](https://en.wikipedia.org/wiki/Sharpe_ratio):

In [4]:
result.bootstrap.conf_intervals

lower     upper
name              conf                     
Log Profit Factor 97.5% -0.762856  1.026865
                  95%   -0.612026  0.869459
                  90%   -0.439416  0.693985
Sharpe Ratio      97.5% -0.161241  0.240268
                  95%   -0.128774  0.206960
                  90%   -0.091338  0.170179

Specifically, **PyBroker** used the [bias corrected and accelerated (BCa) bootstrap method](https://blogs.sas.com/content/iml/2017/07/12/bootstrap-bca-interval.html) to compute the confidence intervals above. The returns used for the bootstrap were sampled per-bar rather than per-trade to maximize the information captured by those metrics.

We can see that the lower bounds of the Sharpe Ratio and log-transformed Profit Factor are both negative. This is not a good sign, and shows that our strategy is not reliably profitable!

## Maximum Drawdown

Still, we continue by looking at bootstrap metrics for maximum drawdown: 

In [5]:
result.bootstrap.drawdown_conf

,amount,percent
conf,,
99.9%,-291212.24,-35.096231
99%,-224718.56,-28.616410
95%,-172629.23,-22.700410
90%,-147709.66,-19.692714


Shown above are the probabilities that the drawdown will not exceed the computed values, which are given in cash amounts and percentages of portfolio equity. Like the Profit Factor and Sharpe Ratio, these confidence levels were computed using per-bar returns obtained from the backtest's out-of-sample results.

The bootstrapped max drawdown of ```-35.09%``` at a ```99.9%``` confidence level is much worse than the ```-7.9%``` we saw in our original results!

Hopefully, this example gives you a sense of the importance of using randomized tests to analyze (and scrutinize) the performance of your trading strategy.

[The next notebook will go over how to use ranking and position sizing in your trading strategies](https://pybroker.com/en/latest/notebooks/4.%20Ranking%20and%20Position%20Sizing.html).